In [1]:
import utils
import torch
import transforms as T
import torchvision

from engine import train_one_epoch, evaluate
from tars.base.dataset import DatasetType
from tars.datasets.instance_segmentation_dataset import InstanceSegmentationDataset
from torchvision.models.detection import maskrcnn_resnet50_fpn


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

In [2]:
dataset_valid_seen = InstanceSegmentationDataset(DatasetType.VAL_SEEN, transforms=get_transform(train=False))
# indices = torch.randperm(256).tolist()
# dataset_valid_seen = torch.utils.data.Subset(dataset_valid_seen, indices)

data_loader_valid_seen = torch.utils.data.DataLoader(
    dataset_valid_seen, batch_size=16, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [3]:
len(dataset_valid_seen)

14965

In [5]:
device = torch.device('cuda:2')
maskrcnn = maskrcnn_resnet50_fpn(num_classes=119)
maskrcnn.eval()
maskrcnn.load_state_dict(torch.load('../moca/weight_maskrcnn.pt'))
maskrcnn = maskrcnn.to(device)

In [7]:
results = evaluate(maskrcnn, data_loader_valid_seen, device=device)

100%|██████████| 14965/14965 [09:19<00:00, 26.75it/s]
creating index...
index created!
Test:  [  0/936]  eta: 0:54:37  model_time: 1.4210 (1.4210)  evaluator_time: 0.8459 (0.8459)  time: 3.5018  data: 1.1971  max mem: 174
Test:  [100/936]  eta: 0:32:27  model_time: 1.3680 (1.3483)  evaluator_time: 0.8772 (0.8717)  time: 2.3318  data: 0.0443  max mem: 174
Test:  [200/936]  eta: 0:28:07  model_time: 1.4334 (1.3390)  evaluator_time: 0.8866 (0.8517)  time: 2.5174  data: 0.0437  max mem: 174
Test:  [300/936]  eta: 0:24:12  model_time: 1.1563 (1.3323)  evaluator_time: 0.6951 (0.8488)  time: 1.9768  data: 0.0445  max mem: 174
Test:  [400/936]  eta: 0:20:21  model_time: 1.2043 (1.3306)  evaluator_time: 0.7726 (0.8486)  time: 2.1566  data: 0.0446  max mem: 174
Test:  [500/936]  eta: 0:16:28  model_time: 1.4695 (1.3221)  evaluator_time: 0.9952 (0.8469)  time: 2.4880  data: 0.0451  max mem: 174
Test:  [600/936]  eta: 0:12:41  model_time: 1.4314 (1.3234)  evaluator_time: 0.9269 (0.8459)  time: 2.5